In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, SpatialDropout2D, ELU
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D
from keras.layers.core import Lambda

from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


### Generator

In [25]:
import os
import csv

reduced = False

if reduced == True:
    csv_filepath = 'data-udacity/driving_log_reduced.csv'
else:
    csv_filepath = 'data-udacity/driving_log.csv'
samples = []
with open(csv_filepath) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
        
def add_to_samples(csv_filepath, samples):
    with open(csv_filepath) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            samples.append(line)
    return samples

samples = add_to_samples('data-extra-laps/driving_log.csv', samples)

samples = add_to_samples('data-recovery/driving_log.csv', samples)

"""
samples = add_to_samples('data-udacity/driving_log_18_bridge_reduced2.csv)
samples = add_to_samples('data-udacity-flipped/driving_csv_flipped_headoff.csv)
"""
samples = samples[1:]
print("Samples: ", len(samples))    
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.1)

Samples:  14112


In [16]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './data-udacity/'+batch_sample[0]
                # name = './data-udacity/IMG/'+batch_sample[0].split('/')[-1]
                center_image = mpimg.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            
            # print("X_train: ", X_train)
            # print("y_train: ", y_train)
            yield shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

### Models

In [9]:
def resize(image):
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 66, 200)

def resize_comma(image):
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 40, 160)

def resize_comma_10_40(image):
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 40, 160)

def halve_size(image):
    imshape = image.shape
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, imshape[0] // 2, imshape[1] // 2)

def resize_80_160(image):
    imshape = image.shape
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 80, 160)

In [20]:
# Comma.ai model
# https://github.com/commaai/research/blob/master/train_steering_model.py

model = Sequential()

# Crop 50 pixels from the top of the image and 20 from the bottom
model.add(Cropping2D(cropping=((70, 25), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

# Resize the data
model.add(Lambda(resize_comma))

model.add(Lambda(lambda x: x[:,:,:,0:1]))

model.add(Lambda(lambda x: (x/255.0) - 0.5))

model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
model.add(ELU())

model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(ELU())

model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))

model.add(Flatten())
# model.add(Dropout(.2))
model.add(ELU())

model.add(Dense(512))
# model.add(Dropout(.5))
model.add(ELU())

model.add(Dense(1))

adam = Adam(lr=0.0001)

model.compile(optimizer=adam, loss="mse", metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_2 (Cropping2D)        (None, 65, 320, 3)    0           cropping2d_input_2[0][0]         
____________________________________________________________________________________________________
lambda_4 (Lambda)                (None, 40, 160, 3)    0           cropping2d_2[0][0]               
____________________________________________________________________________________________________
lambda_5 (Lambda)                (None, 40, 160, 1)    0           lambda_4[0][0]                   
____________________________________________________________________________________________________
lambda_6 (Lambda)                (None, 40, 160, 1)    0           lambda_5[0][0]                   
___________________________________________________________________________________________

Data preprocessing reference: [Geoff Breemer](https://carnd-forums.udacity.com/questions/36045049/answers/36047341)

### Train model

In [21]:
# Train model
batch_size = 32
nb_epoch = 20
# data_augmentation = True
 
checkpointer = ModelCheckpoint(filepath="./tmp/comma-v2f.{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1, save_best_only=False)
    
model.fit_generator(train_generator, 
                    samples_per_epoch=len(train_samples), 
                    validation_data=validation_generator,
                    nb_val_samples=len(validation_samples), nb_epoch=nb_epoch,
                    callbacks=[checkpointer])

Epoch 1/20
7232/7232 [==============================] - 35s - loss: 0.0172 - acc: 0.5415 - val_loss: 0.0157 - val_acc: 0.5349
Epoch 2/20
7232/7232 [==============================] - 38s - loss: 0.0176 - acc: 0.5431 - val_loss: 0.0150 - val_acc: 0.5557
Epoch 3/20
7232/7232 [==============================] - 46s - loss: 0.0169 - acc: 0.5415 - val_loss: 0.0165 - val_acc: 0.5264
Epoch 4/20
7232/7232 [==============================] - 47s - loss: 0.0174 - acc: 0.5431 - val_loss: 0.0144 - val_acc: 0.5569
Epoch 5/20
7232/7232 [==============================] - 43s - loss: 0.0168 - acc: 0.5415 - val_loss: 0.0159 - val_acc: 0.5322
Epoch 6/20
7232/7232 [==============================] - 48s - loss: 0.0174 - acc: 0.5431 - val_loss: 0.0144 - val_acc: 0.5661
Epoch 7/20
7232/7232 [==============================] - 47s - loss: 0.0168 - acc: 0.5415 - val_loss: 0.0156 - val_acc: 0.5248
Epoch 8/20
7232/7232 [==============================] - 52s - loss: 0.0173 - acc: 0.5431 - val_loss: 0.0140 - val_acc:

In [22]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model-comma-v2f.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


### Deprecated code